In [1]:
import numpy as np
import random
from collections import Counter

In [2]:
ANNA_PATH = 'corpora/AnnaKarenina.txt'
WNP_ENG_PATH = 'corpora/WarAndPeaceEng.txt'
WNP_PATH = 'corpora/WarAndPeace.txt'

ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

#### 1.Реализуйте базовый частотный метод по Шерлоку Холмсу:
#### - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
#### - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
#### - расшифруйте их таким частотным методом.


In [3]:
# Файл "Анна Каренина" имеет больше символов, поэтому используем его в качестве обучающего
with open(ANNA_PATH, 'r', encoding='utf-8') as f:
    anna = f.read()

with open(WNP_PATH, 'r', encoding='utf-8') as f:
    wnp = f.read()

with open(WNP_ENG_PATH, 'r', encoding='utf-8') as f:
    wnp_eng = f.read()

print(f' Anna len = {len(anna)}, WNP len = {len(wnp)}, WNP_ENG len = {len(wnp_eng)}')

 Anna len = 1813200, WNP len = 717873, WNP_ENG len = 3226615


Возьмем "Анну Каренину" в качестве тренировочного тектса, тк там банально больше статистики.

In [6]:
train_text = anna.lower()
test_text = wnp.lower()

# Возьмем несколько предложений для кодирования
start, end = 34804, 35103
test_text = "".join([i for i in test_text[start:end] if i in ALPHABET])
print(f'Тестовый текст:"{test_text}"')

Тестовый текст:"анна михайловна  сказал он с своею всегдашнею фамильярностью и скукой в голосе  для меня почти невозможно сделать то что вы хотите но чтобы доказать вам как я люблю вас и чту память покойного отца вашего я сделаю невозможное сын ваш будет переведен в гвардию вот вам моя рука довольны вы"


In [7]:
# Случайным образом переставим буквы в тексте.

def encode(text):
    cur_text_alphabet = ''.join(set(text))
    сypher = ''.join(random.sample(ALPHABET, len(cur_text_alphabet)))
    encoded_text = text.lower().translate(str.maketrans(cur_text_alphabet, сypher))
    return ''.join([char for char in encoded_text if char in сypher])


encoded_test_text = encode(test_text)
print(f'Зашифрованный тестовый текст: "{encoded_test_text}", \nДлина текста = {len(encoded_test_text)}')

Зашифрованный тестовый текст: "ьяяьхз жьцъыйяьххищьльъхыяхихийывухйивапьоявухдьз ъркфяыиерух хищбщыцхйхаыъыивххпъкхзвякхшыче хявйылзыюяыхипвъьерхеыхчеыхйгхжые евхяыхчеытгхпыщьльерхйьзхщьщхкхъутъухйьих хчебхшьзкерхшыщыцяыаыхыесьхйьоваыхкхипвъьухявйылзыюяывхигяхйьохтбпвехшвфвйвпвяхйхайьфп ухйыехйьзхзыкхфбщьхпыйыърягхйг", 
Длина текста = 287


In [8]:
# Также зададим функцию для подсчета посимвольной точности перевода
def decoding_accuracy(true_test, pred_text):
    accuracy = 0
    for i, j in zip(true_test, pred_text):
        if i == j:
            accuracy += 1
    return accuracy / len(true_test)

In [9]:
# Частотный декодер
class FreqDecoder:

    def __init__(self, n_gram_len):
        self.n_gram_len = n_gram_len

    def fit(self, train_text):
        train_text = ''.join([char for char in train_text if char in ALPHABET])
        splitted_text = [train_text[i:i + self.n_gram_len] for i in range(len(train_text) - self.n_gram_len + 1)]
        # Подсчитываем кол-во встречаемости n-граммы в тренировочном тексте
        self.frequency = Counter(splitted_text)
        #Сортируем по убыванию
        sorted_idx = np.argsort(list(self.frequency.values()))[::-1]
        self.frequency_sorted = np.array(list(self.frequency.keys()))[sorted_idx]

    def decode(self, text):
        #Повторяем аналогично обучению, но для зашифрованного текста
        original_len = len(text)
        splitted_text = [text[i:i + self.n_gram_len] for i in
                         range(0, len(text) - self.n_gram_len + 1, self.n_gram_len)]
        encoded_frequency = Counter(splitted_text)
        sorted_idx = np.argsort(list(encoded_frequency.values()))[::-1]
        encoded_frequency_sorted = np.array(list(encoded_frequency.keys()))[sorted_idx]
        decoder_size = min(len(encoded_frequency_sorted), len(self.frequency_sorted))
        # Задаем словарь перевода частот
        decoder = {}
        for i in range(decoder_size):
            decoder[encoded_frequency_sorted[i]] = self.frequency_sorted[i]
        # Декодируем текст
        decoded_text = ''.join([decoder[n_gram] for n_gram in splitted_text])[:original_len]
        return decoded_text

In [10]:
decoder_unigram = FreqDecoder(n_gram_len=1)
decoder_unigram.fit(train_text)

In [11]:
# Декодируем закодированный текст униграммным частотным декодером
decoded_text = decoder_unigram.decode(encoded_test_text)
print(f'Настоящий текст: ')
print(test_text)
print('Декодированный текст: ')
print(decoded_text)

print('\nТочность частотного декодирования униграмми: ', decoding_accuracy(test_text, decoded_text))

Настоящий текст: 
анна михайловна  сказал он с своею всегдашнею фамильярностью и скукой в голосе  для меня почти невозможно сделать то что вы хотите но чтобы доказать вам как я люблю вас и чту память покойного отца вашего я сделаю невозможное сын ваш будет переведен в гвардию вот вам моя рука довольны вы
Декодированный текст: 
еиие рдхешлоаие  скеыел ои с саонм аснявейинм эердлпугиостпм д скбкош а яолосн  влу рниу чозтд инаоыроюио свнлетп то зто аь хотдтн ио зтожь вокеыетп аер кек у лмжлм аес д зтб черутп чокошиояо отще аейняо у свнлем инаоыроюион сьи аей жбвнт чнгнанвни а яаегвдм аот аер роу гбке воаолпиь аь

Точность частотного декодирования униграмми:  0.43902439024390244


#### 2.Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
#### - подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
#### - проведите тестирование аналогично п.1, но при помощи биграмм.

In [12]:
decoder_bigram = FreqDecoder(n_gram_len=2)
decoder_bigram.fit(train_text)

In [13]:
# Декодируем закодированный текст биграммным частотным декодером
decoded_text = decoder_bigram.decode(encoded_test_text)

print(f'Настоящий текст: ')
print(test_text)
print('Декодированный текст: ')
print(decoded_text)

print('\n Точность частотного декодирования биграммами: ', decoding_accuracy(test_text, decoded_text))

Настоящий текст: 
анна михайловна  сказал он с своею всегдашнею фамильярностью и скукой в голосе  для меня почти невозможно сделать то что вы хотите но чтобы доказать вам как я люблю вас и чту память покойного отца вашего я сделаю невозможное сын ваш будет переведен в гвардию вот вам моя рука довольны вы
Декодированный текст: 
льотни браовм л е лакаво че е ентьо коол норрей прнелилен в пое ва о меллоовкоосроалу заи ерон с вго к та носты ь  пна по  иазонбы са ин у ия лака зо тожеемес ртиито допонаеги тоась ат оакода  ддао  нтеа алността с вго к т  е омо  нанк оги ск ветдео илрисечто  до тонит вевил я енне ео 

 Точность частотного декодирования биграммами:  0.09407665505226481


Точность такого декодирования оказалась сильно ниже, чем при униграммном подходе. Но кажется что точность может быть лучше на бОльшем объеме тестового текста. Проверим:

In [27]:
encoded_test_big_text = encode(''.join([i for i in wnp.lower() if i in ALPHABET]))
decoded_text = decoder_bigram.decode(encoded_test_big_text)
print('\n Точность частотного декодирования биграммами, на всем произведении: ', decoding_accuracy(wnp.lower(), decoded_text))
decoded_text = decoder_unigram.decode(encoded_test_big_text)
print('\n Точность частотного декодирования униграммами, на всем произведении: ', decoding_accuracy(wnp.lower(), decoded_text))


 Точность частотного декодирования биграммами, на всем произведении:  0.05404159231507523

 Точность частотного декодирования униграммами, на всем произведении:  0.05469073220472145


Действительно, при таком подходе - точность оценки методом на биграммах на сопоставимом уровне с униграммным методом.

#### Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
#### - предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
#### - реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [28]:
class MCMCDecoder:
    def __init__(self):
        self.alphabet = list(ALPHABET)
        self.alphabet_len = len(self.alphabet)
        self.char_index = dict(zip(self.alphabet, range(self.alphabet_len)))

    def fit(self, text):
        text = ''.join([char for char in text.lower() if char in self.alphabet])
        # построим квадратную матрицу где столбцы и строки - символы алфавита.
        proposition_map = np.zeros((self.alphabet_len, self.alphabet_len))
        for text_idx in range(1, len(text)):
            cur_idx = self.char_index[text[text_idx - 1]]
            cur_idx_next = self.char_index[text[text_idx]]
            proposition_map[cur_idx, cur_idx_next] += 1
        proposition_map = np.clip(proposition_map, a_min=1, a_max=None)
        self.proposition_map = proposition_map / proposition_map.sum(axis=1)[:, np.newaxis]
        self.log_prop_map = np.log(self.proposition_map)

    def swap(self, tokens, idx1, idx2):
        tokens[idx1], tokens[idx2] = tokens[idx2], tokens[idx1]
        return tokens

    def decoder(self, text, tokens):
        decoder_dict = dict(zip(tokens, self.alphabet))
        return ''.join(map(lambda x: decoder_dict[x], text))

    def log_text_score(self, text, shuffled_tokens):
        text = ''.join([char for char in text.lower() if char in self.alphabet])
        text = self.decoder(text, shuffled_tokens)
        log_score = 0
        for text_idx in range(1, len(text)):
            cur_idx = self.char_index[text[text_idx - 1]]
            cur_idx_next = self.char_index[text[text_idx]]
            log_score += self.log_prop_map[cur_idx, cur_idx_next]
        return log_score

    def random_index(self, max_val):
        idx1 = 0
        idx2 = 0
        while idx1 == idx2:
            idx1 = random.randint(0, max_val)
            idx2 = random.randint(0, max_val)
        return idx1, idx2

    def decode(self, text, n_iters):
        text = ''.join([char for char in text.lower() if char in self.alphabet])
        shuffled_tokens = self.alphabet.copy()
        random.shuffle(shuffled_tokens)
        self.decoded_tokens = shuffled_tokens.copy()
        max_log_score = log_score = self.log_text_score(text, shuffled_tokens)
        for i in range(n_iters):
            idx1, idx2 = self.random_index(self.alphabet_len - 1)
            shuffled_tokens = self.swap(shuffled_tokens, idx1, idx2)
            new_log_score = self.log_text_score(text, shuffled_tokens)
            # Выбираем следующий токен по алгоритму Метрополиса-Гастингса
            if new_log_score <= log_score:
                if np.log(random.uniform(0, 1)) > new_log_score - log_score:
                    shuffled_tokens = self.swap(shuffled_tokens, idx1, idx2)
                else:
                    log_score = new_log_score
            else:
                log_score = new_log_score
                if new_log_score > max_log_score:
                    max_log_score = new_log_score
                    self.decoded_tokens = shuffled_tokens.copy()
        return self.decoder(text, self.decoded_tokens)

In [15]:
# Создаем МСМС декодер и обучаем его
mcmc_decoder = MCMCDecoder()
mcmc_decoder.fit(train_text)

In [21]:
# Декодируем текст с помощью МСМС
best_iter = 0
best_acc = 0
best_decoded_text = None

for i in range(0, 100000, 10000):
    decoded_text = mcmc_decoder.decode(encoded_test_text, n_iters=i)
    curr_acc = decoding_accuracy(test_text, decoded_text)
    print(f'Количество итераций {i}, точность расшифровки - {decoding_accuracy(test_text, decoded_text)}')
    if curr_acc > best_acc:
        best_acc = curr_acc
        best_iter = i
        best_decoded_text = decoded_text


print(f'Настоящий текст: ')
print(test_text)
print('Декодированный текст: ')
print(best_decoded_text)
print(f'Итоговая точность на {best_iter} итераций - {best_acc}')

Количество итераций 0, точность расшифровки - 0.11846689895470383
Количество итераций 10000, точность расшифровки - 0.26480836236933797
Количество итераций 20000, точность расшифровки - 0.0627177700348432
Количество итераций 30000, точность расшифровки - 0.8432055749128919
Количество итераций 40000, точность расшифровки - 0.9372822299651568
Количество итераций 50000, точность расшифровки - 0.9372822299651568
Количество итераций 60000, точность расшифровки - 0.9372822299651568
Количество итераций 70000, точность расшифровки - 0.2264808362369338
Количество итераций 80000, точность расшифровки - 0.7909407665505227
Количество итераций 90000, точность расшифровки - 0.9372822299651568
Настоящий текст: 
анна михайловна  сказал он с своею всегдашнею фамильярностью и скукой в голосе  для меня почти невозможно сделать то что вы хотите но чтобы доказать вам как я люблю вас и чту память покойного отца вашего я сделаю невозможное сын ваш будет переведен в гвардию вот вам моя рука довольны вы
Декоди

Итого: качество MCMC декодера - в разы лучше частотного метода.
Однако, алгоритм не воспроизводимый, а также желательно подбирать оптимальное количество итераций МГ, на котором в среднем работает неплохо.

#### 4.Расшифруйте сообщение:

In [22]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
print('Зашифрованное сообщение:')
print(message)
encoded_message = encode(message)
print('\nЗашифрованное сообщение перекодированное случайным образом на кириллицу:')
print(encoded_message)

Зашифрованное сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

Зашифрованное сообщение перекодированное случайным образом на кириллицу:
э шмцфжцфмнмяэцюлчкбшщюжтцмшмцвлрямцюлчкбшщюжтцяэс яцицдялхлц лльпэюмацслялчжтцшэхслцвчлрмябящц слчээцф эхлцфжцф эц нэшбшмцвчбфмшщюлцмцвлширмяэцкбс мкбшщюжтцьбшшцйбцвл шэнюээцрэяфэчялэцйбнбюмэцсич бцуляацслюэрюлцацюмрэхлцюэцльэпбз


In [23]:
# Декодируем текст с помощью частотного метода
message_decoded_baseline = decoder_bigram.decode(encoded_message)
print('Сообщение, декодированное частотным методом: ')
print(message_decoded_baseline)

Сообщение, декодированное частотным методом: 
емпоонкоь канигоал кнеосл во впрть вна ч сст нот брам рой  ии  деновлая  оал н ттое о у  мелили е риа сето пчтонт и ведапоо одь ненаако аты аза  кбыин сст н у с зжео деетологваорв н залиреле ея ерскте оное та   пнооман игоа к вило


In [24]:
mcmc_decoder = MCMCDecoder()
#Используем два корпуса текстов, на всякий случай :)
mcmc_decoder.fit(train_text + wnp.lower())

In [25]:
for i in range(0, 100000, 10000):
    decoded_message = mcmc_decoder.decode(encoded_message, n_iters=i)
    print(f'Итерация {i}: \n {decoded_message} \n')

Итерация 0: 
 фбяйплгплйсймфпъныжаяюъг пйяйпонтмйпъныжаяюъг пмфибмпдпрмнэнпбннзшфъйёпинмныг пяфэинпоынтймамюпбиныффплбфэнплгплбфпбсфяаяйпоыалйяюънпйпонядтймфпжаибйжаяюъг пзаяяпхапонбяфсъффптфмлфымнфпхасаъйфпидыбапкнмёпинъфтънпёпъйтфэнпъфпнзфшав 

Итерация 10000: 
 ндим сл смьмен оваприхолю мим квтем оваприхолю енуде ч жевыв дввзгномя увевалю иныув кавтмерех дуванн сдныв сл сдн дьнирим карсмихов м квичтмен прудмприхолю зрии бр квдиньонн тнеснаевн брьромн учадр цвея увонтов я омтныв он взнгрш 

Итерация 20000: 
 ивна ду дазали мергонымую ана песла мергонымую литвл к элече веебцимая телерую ничте пресалолы втерии двиче ду дви взинона проданыме а пенксали готвагонымую бонн хо певнизмии силдирлеи хозомаи ткрво желя темисме я масиче ми ебицой 

Итерация 30000: 
  датнялняткто ниевырамильнтатнцесотниевырамильно удонзнгоепендееюб итчнуеоевльна пуенцвесторомндуев  няд пенялняд ндк аратнцврятамиентнцеазсто нырудтырамильнюраанфрнцеда ки  нс оя вое нфркрит нузвдрнщеочнуеи сиенчнитс пени нею брж 



Лучшая попытка расшифровки на 40000 итераций:
"если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж"

Мое предположение по "дошифровке":
"если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максидальный балл за послелнее четвертое задание курса хотя конечно я ничего не обещаю"

Достаточно двух перестановок, чтобы дошифровать:
д <-> м
ж <-> ю